## Imports

In [265]:
from zipfile import ZipFile
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os
import re

## Data Preparation

In [266]:
def read_images(filename,preprocess = True):
    
    filename = filename
    
    imgs = []
    
    img_names = []
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    
    with ZipFile(filename) as archive:
        for count, entry in enumerate(archive.infolist()):
            with archive.open(entry) as file:
                if(file.name.endswith("jpg") or file.name.endswith("png")):
                    
                    img = np.array(Image.open(file).convert('L'))

                    if(preprocess):
               
                        img = clahe.apply(img)

                    imgs.append(img)
                    
                    img_names.append(file.name)

    return imgs,img_names

In [267]:
template_imgs = []

template_img_names = []

test_imgs = []

test_img_names = []

for item in os.listdir():
    
    if(item.endswith("zip")):
        
        if(item != "Test.zip"):
            
            tmp,tmp1 = read_images(item)

            template_imgs += tmp

            template_img_names += tmp1

        else :

            tmp,tmp1 = read_images(item)

            test_imgs += tmp

            test_img_names += tmp1
        
test_imgs = np.array(test_imgs)
template_imgs = np.array(template_imgs)

In [268]:
test_imgs.shape,template_imgs.shape

((24, 250, 250), (96, 250, 250))

## Minimum Pair Distance (MDP)

In [269]:
def MDP(img1, img2):

    sift = cv2.xfeatures2d.SIFT_create()
    
    kp1, des1 = sift.detectAndCompute(img1,None)

    kp2, des2 = sift.detectAndCompute(img2,None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Match descriptors.
    matches = bf.match(des1,des2)

    min_dist = np.min([matches[i].distance for i in range(len(matches))])
    
#     matches = sorted(matches, key = lambda x:x.distance)

# # Draw first 10 matches.
#     img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches, img1)

#     plt.imshow(img3),plt.show()

    return min_dist

## Regular Grid (RG)

In [270]:
def RG(img1, img2):
    
    x,y = img1.shape
    
    x = np.int(x/4)
    
    y = np.int(y/2)
    
    i = 0
    
    min_dist = []
    
    while(x+i < img1.shape[0]):
        
        j = 0
        
        while(y+j < img1.shape[1]):
            
            min_dist.append(MDP(img1[i:x + i,j:y+j],img2[i:x + i,j:y+j]))
            
            
            j = np.int(j + 0.75*y)
            
        i = np.int(i +0.75*x)    
            
    return np.mean(min_dist)
    

## Evaluation

In [271]:
threshold = 30
good_matches = 0
bad_matches = 0
for img,img_name in zip(test_imgs,test_img_names):
    
    for i,name in zip(template_imgs,template_img_names):
        
        if((MDP(img,i) <= 30 and img_name.split("/")[-1] == name.split("/")[-1]) or (MDP(img,i) > 30 and img_name.split("/")[-1] != name.split("/")[-1])):
            good_matches +=1
            
        else:
#             print(img_name,name,MDP(img,i))
            bad_matches +=1
            

In [272]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9895833333333334, 0.010416666666666666)

In [273]:
# [i for i in range(len(test_img_names)) if test_img_names[i] == "Image2/Test/Aaron_Peirsol_0002.jpg"]

In [274]:
# MDP(test_imgs[18],template_imgs[14]),MDP(test_imgs[18],template_imgs[0])

In [275]:
good_matches,bad_matches

(2280, 24)